In [ ]:
import matplotlib.pyplot as plt

from typing import List
import h5py
import numpy as np
import tensorflow as tf
import h5py
import os
from tensorflow.python.keras.utils.np_utils import to_categorical
from keras.src.backend.tensorflow.sparse import zeros_int8
from tensorflow import Tensor

In [ ]:
#må byttes med der du har lagret filene
file_path = "C:\\Users\\Kaspar\\ChipWhisperer5_64\\cw\\home\\portable\\chipwhisperer\\jupyter\\courses\\sca101\\training_set_new_kpts.hdf5"
with h5py.File(file_path, "r") as h5_file:
    plaintext_set = h5_file['data'][:]
    trace_set = h5_file['trace'][:]
    key_set = h5_file['key'][:]
    sub_byte_out_set = h5_file['sub_byte_out'][:]

This is one of the few points where trace is manipulated, and the first place we will look for changes. Therefore we create a new array to contain in so we can always compare with the original

In [ ]:
plt.plot(trace_set[0])
plt.show()

In [ ]:
scaled_trace_set = tf.keras.layers.Rescaling(1. / 127.5, offset=-1)(trace_set)
scaled_trace_set = tf.expand_dims(scaled_trace_set, axis=-1)

In [ ]:
plt.plot(scaled_trace_set[0])
plt.show()

In [ ]:
h5_file.close()

In [ ]:
trace_set.shape

In [ ]:
scaled_trace_set.shape

In [ ]:
import matplotlib.pyplot as plt
plt.plot(trace_set[0][35:45])
plt.plot(trace_set[9][35:45])


In [ ]:
#the numerical difference between the two traces on point 42:
diff = trace_set[0][42] - trace_set[9][42]
print(trace_set[0][42])
print(trace_set[9][42])
print(diff)

In [ ]:
#testing if it holds through scaling
scaled_diff = scaled_trace_set[0][42] - scaled_trace_set[9][42]
print(scaled_trace_set[0][41:44])
print(scaled_trace_set[9][41:44])
print(scaled_diff)

In [ ]:
plt.plot(scaled_trace_set[0][35:45])
plt.plot(scaled_trace_set[9][35:45])
plt.plot(scaled_trace_set[5][35:45])
print(scaled_trace_set[0][35:45])
print(scaled_trace_set[5][35:45])
print(plaintext_set[5])


looks pretty freaking close

First we create a dataset out of it, like i have done previously

In [ ]:
num_traces = len(trace_set)
num_bytes = 16
sub_byte_in_set = np.zeros((num_traces, 16), dtype=np.uint8)

for i in range(num_traces):
    for byte_index in range(num_bytes):
        pts=plaintext_set[i][byte_index] ^ key_set[i,byte_index]
        sub_byte_in_set[i,byte_index] = pts
#check if this works
sub_byte_in_set = sub_byte_in_set

In [ ]:
sub_byte_out = np.transpose(sub_byte_out_set, (1, 0))
sub_byte_in = np.transpose(sub_byte_in_set, (1, 0))
key = np.transpose(key_set, (1, 0))

In [ ]:
num_shards = 1
num_traces_per_shard = 10
file_path = "\\...\\DAT255_SCA\\datasets"

#index to mark start and stop for slicing
start_index = 0
stop_index = num_traces_per_shard
dataset_name = "troubleshooting" #change depending on the set you create

#create a new h5py file that will contain the shards
f = h5py.File(f"{dataset_name}.hdf5", "w")

#Loop trough the dataset, creating groups for every key
for i in range(num_shards):
# create a group for each key, the name of the group will be the key in hex. This is refering to the original key
#array, different indexing is needed if you want to use key
    group_name = key_set[start_index].tobytes().hex()
#Create one group representing a shard

    group = f.create_group(group_name, track_order=True)
#oppretter datasettene
#rad, kolonne, steg vi vil ha alle rader, 256 kolonner av gangen. Også vurder å ta to categorical senere i prosessen
#Må treffe på fordelingen her
    group.create_dataset("traces", data = scaled_trace_set[start_index:stop_index, :, :])
    group.create_dataset("key", data = key[:, start_index:stop_index])
    group.create_dataset("sub_bytes_in", data = sub_byte_in[:, start_index:stop_index])
    group.create_dataset("sub_bytes_out", data = sub_byte_out[:, start_index:stop_index])
#må også ha trace slik at den kan hentes
    start_index += num_traces_per_shard
    stop_index += num_traces_per_shard
f.close()

When this is done we have one shard with 10 traces.

In [ ]:
f = h5py.File(f"{dataset_name}.hdf5", "r")
i=0
for group in f.keys():
    print(group)
    i+=1
    print(i)
    print()
    for dset in f[group].keys():
        print(dset)
f.close()

In [ ]:
shard = []
attack_byte = 0
attack_point = "sub_bytes_in"
max_trace_length = 5000
num_traces = 10
full_key = False
x_List: List[Tensor] = []
y_List: List[Tensor] = []

#need to open the file to read any info
with (h5py.File(f"{dataset_name}.hdf5", "r")) as f:
    for group in f.keys():
        group_name = f[group]

        x_shard = group_name["traces"][:num_traces_per_shard, :5000, :]
        x_shard = tf.convert_to_tensor(x_shard, dtype="float32")

        y_shard = group_name["sub_bytes_in"][attack_byte]
        y_shard = y_shard[:num_traces_per_shard]
        y_shard = tf.keras.utils.to_categorical(y_shard, 256)
        y_shard = tf.convert_to_tensor(y_shard, dtype="uint8")


        x_List.append(x_shard)
        y_List.append(y_shard)
    x: Tensor = tf.concat(x_List, axis=0)
    y: Tensor = tf.concat(y_List, axis=0)

In [ ]:
x_train = x
y_train = y

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
plt.figure(figsize=(50, 40))
plt.plot(x_train[0][40:45], label="Trace 0",linewidth=2.5, color="black")
#plt.plot(x_train[1][40:45], label="Trace 1",linewidth=2.5)
#.plot(x_train[2][40:45], label="Trace 2",linewidth=2.5)
#plt.plot(x_train[3][40:45], label="Trace 3",linewidth=2.5)
plt.plot(x_train[4][40:45], label="Trace 4",linewidth=2.5)
plt.plot(x_train[5][40:45], label="Trace 5",linewidth=2.5)
plt.plot(x_train[6][40:45], label="Trace 6",linewidth=2.5)
plt.plot(x_train[7][40:45], label="Trace 7",linewidth=2.5)
plt.plot(x_train[8][40:45], label="Trace 8",linewidth=2.5)
plt.plot(x_train[9][40:45], label="Trace 9",linewidth=2.5, color="red")

plt.title("traces", fontsize=20)
plt.legend(fontsize=30, loc='upper left', bbox_to_anchor=(1, 1))
plt.tick_params(axis='both', which='major', labelsize=14, width=2, length=6)  # Tykkere ticks
plt.tick_params(axis='both', which='minor', length=4, width=1.5)

plt.show()

Vi kan se at trace 9, hvor verdien er 255 er det øverste, men det er faktisk ikke trace 0 hvor verdien er 0 som er det laveste.

Vi kan skrive ut byte 0 i hvert trace, og se om det er sammenheng mellom tall og plassering


In [ ]:
#Siden denne er gjort kategorisk går det selvfølgelig ikke, og vi må heller se på plaintext
for i in range(len(plaintext_set)):
    print("Plaintext: ", i, plaintext_set[i][:1])